In [ ]:
################################
# Quantum  algorithm for estimating the determinant
# 1 MAY 2025 by Vittorio Giovannetti, Seth Lloyd, Lorenzo Maccone

In [ ]:
#Tests
def test_matrices():
    [[3.0, 0.5], [0.5, 2.5]]     # κ ≈ 1.5, excellent , recommended
    [[2.0, 0.3], [0.3, 1.5]]     # κ ≈ 1.55, very stable ,Recommended
    [[1.0, 0.9], [0.9, 1.0]]     # κ = 19, ill-conditioned
    [[5.0, 0.5], [4.0, 0.4]]     # K = Large Number System fails due to lack in QRAM
    [[5.0, 0.5], [0.4, 4.0]]     # K = 1.35
    return []

In [ ]:
import numpy as np
import scipy.linalg
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import transpile
from qiskit.circuit.library import QFT # Quantum Fourier Transform
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from IPython.display import display
import matplotlib.pyplot as plt

In [ ]:
class QuantumDeterminantEstimator:
    def __init__(self, matrix, target_error=0.2):
        self.original_matrix = np.array(matrix, dtype=complex)
        self.n = len(matrix)
        self.target_error = target_error
        
        # Check matrix properties
        eigenvalues = np.linalg.eigvals(self.original_matrix)
        print(f"Original eigenvalues: {eigenvalues}")
        
        # Make Sure its postive
        if np.any(np.real(eigenvalues) <= 0):
            print("Converting to positive definite matrix") #(A^H @ A + εI)
            self.original_matrix = self.original_matrix.conj().T @ self.original_matrix + 0.1 * np.eye(self.n)
            eigenvalues = np.linalg.eigvals(self.original_matrix)
            print(f"New eigenvalues: {eigenvalues}")
        
        #λ_max = 1/2, λ_min = 1/(2κ)
        self.lambda_max = np.max(np.real(eigenvalues))
        self.lambda_min = np.min(np.real(eigenvalues))
        self.condition_number = self.lambda_max / self.lambda_min #K
        
        # Rescale matrix
        self.matrix = self.original_matrix / (2 * self.lambda_max)
        self.rescaled_eigenvalues = np.real(eigenvalues) / (2 * self.lambda_max)
        
        # Calculate parameters
        self.mu = np.mean(np.log(self.rescaled_eigenvalues))  # μ = (1/n)Σlog(λ_j)
        self.delta = np.std(np.log(self.rescaled_eigenvalues))  # Δ = std(log(λ_j))
        
        helper= abs(self.mu) * target_error #|μ|ε
        
        # Set precision using: 2^m = 4κ/(|μ|ε)
        self.precision_bits = max(int(np.ceil(np.log2(4 * self.condition_number / helper))), 4)
        
        # Set samples using N_mc = (2Δ/(|μ|ε))²
        self.num_samples = max(int(np.ceil((2 * self.delta /np.power(helper,2)))), 100)
        self.num_qubits = max(int(np.ceil(np.log2(self.n))), 1)
        self._print_setup()
    
    def _print_setup(self):
        print(f"\n=== Quantum Determinant Estimator Setup ===")
        print(f"Matrix size: {self.n}×{self.n}")
        print(f"System qubits: {self.num_qubits}")
        print(f"Original λ_max: {self.lambda_max:.6f}, λ_min: {self.lambda_min:.6f}")
        print(f"Condition number κ: {self.condition_number:.2f}")
        print(f"Rescaled eigenvalues: {self.rescaled_eigenvalues}")
        print(f"μ (mean log λ): {self.mu:.6f}")
        print(f"Δ (std log λ): {self.delta:.6f}")
        print(f"Precision bits m: {self.precision_bits}")
        print(f"Monte Carlo samples: {self.num_samples}")

In [ ]:
def build_qpe_circuit(self):
    
    # Initialize Quantum Registers and Circuit
    precision_reg = QuantumRegister(self.precision_bits, 'prec') #m qubits for precision
    system_reg = QuantumRegister(self.num_qubits, 'sys') #n qubits for system
    classical_reg = ClassicalRegister(self.precision_bits, 'c') #m bits for measurement
    qc = QuantumCircuit(precision_reg, system_reg, classical_reg) #init circuit
    
    #Initialize precision register in superposition
    for i in range(self.precision_bits):
        qc.h(precision_reg[i])
    
    #Initialize system with equal superposition (|ψ> = 1/√n Σ|j>)
    for i in range(self.num_qubits):
        qc.h(system_reg[i])
    
    qc.barrier(label="Initialization")
    
    #Controlled unitary operations U^(2^j)
    for j in range(self.precision_bits):
        # Controlled evolution
        for idx, eigenval in enumerate(self.rescaled_eigenvalues[:min(len(self.rescaled_eigenvalues), np.power(2, self.num_qubits))]):
            phase = 2 * np.pi * eigenval * (np.power(2, j)) # phase = 2πλ_j * 2^j
            target_qubit = idx % self.num_qubits
            qc.cp(phase, precision_reg[j], system_reg[target_qubit])
    qc.barrier(label="Controlled Evolution")
    
    #IQFT on precision register
    qft = QFT(self.precision_bits, inverse=True, do_swaps=True).to_gate(label="QFT†")
    qc.append(qft, precision_reg)
    
    #Measurment
    qc.barrier(label="Measurement")
    qc.measure(precision_reg, classical_reg)
    
    return qc

QuantumDeterminantEstimator.build_qpe_circuit = build_qpe_circuit

In [ ]:
def process_qpe_results(self, counts): #Process QPE results to extract eigenvalue samples
    
    eigenvalue_samples = [] #List to hold eigenvalue estimates
    
    for bitstring, count in counts.items():
        
        # Convert bitstring to phase estimate
        phase_int = int(bitstring, 2)   
        estimated_phase = phase_int / (2**self.precision_bits)
        
        # Handle phase wrapping
        if estimated_phase > 0.5:
            estimated_phase = estimated_phase - 1.0
        
        # Convert phase to eigenvalue estimate
        if abs(estimated_phase) > 1e-12:
            # From QPE theory: phase ≈ eigenvalue for our rescaled problem
            eigenvalue_estimate = abs(estimated_phase) * 2  # Rescale back
            if eigenvalue_estimate > 1e-12:
                eigenvalue_samples.extend([eigenvalue_estimate] * count)
                
    return eigenvalue_samples

def estimate_log_determinant(self, eigenvalue_samples): #Estimate log determinant using Monte Carlo formula
    if not eigenvalue_samples:
        return None, float('inf')
    
    # α_est = (n/N_mc) * Σ log(λ_ℓ) (From paper)
    valid_samples = [e for e in eigenvalue_samples if e > 1e-12]
    
    if not valid_samples:
        return None, float('inf')
    
    # Monte Carlo estimate
    log_samples = [np.log(e) for e in valid_samples]
    N_mc = len(valid_samples)
    alpha_estimate = (self.n / N_mc) * sum(log_samples)
    
    # Convert back to original scale
    original_log_det_estimate = alpha_estimate + self.n * np.log(2 * self.lambda_max)
    
    # True value for comparison
    true_det = np.linalg.det(self.original_matrix)
    true_log_det = np.log(abs(true_det)) if abs(true_det) > 1e-12 else -np.inf
    
    # Calculate relative error
    if abs(true_log_det) > 1e-12:
        relative_error = abs(original_log_det_estimate - true_log_det) / abs(true_log_det)
    else:
        relative_error = np.inf
    
    return original_log_det_estimate, relative_error

QuantumDeterminantEstimator.process_qpe_results = process_qpe_results
QuantumDeterminantEstimator.estimate_log_determinant = estimate_log_determinant

In [ ]:
def run_corrected_simulation(self, shots=None):
    if shots is None:
        shots = min(self.num_samples * 10, 4096)
    
    print(f"\n=== Simulation ===")
    print(f"Shots: {shots}")
    
    # Build and run QPE circuit
    qc = self.build_qpe_circuit()
    print(f"Circuit depth: {qc.depth()}")
    print(f"Circuit gates: {len(qc.data)}")
    
    # Simulation using AER Simulator
    backend = AerSimulator()
    transpiled_qc = transpile(qc, backend)
    job = backend.run(transpiled_qc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    print(f"Raw measurement counts: {dict(list(counts.items())[:10])}...")
    eigenvalue_samples = self.process_qpe_results(counts)
    log_det_estimate, relative_error = self.estimate_log_determinant(eigenvalue_samples)
    
    # Results
    true_det = np.linalg.det(self.original_matrix)
    true_log_det = np.log(abs(true_det))
    print(f"\n=== Results ===")
    print(f"Eigenvalue samples collected: {len(eigenvalue_samples)}")
    if eigenvalue_samples: #To avoid error if empty
        print(f"Mean estimated eigenvalue: {np.mean(eigenvalue_samples):.6f}")
        print(f"True mean eigenvalue: {np.mean(np.real(np.linalg.eigvals(self.original_matrix))):.6f}")
        print(f"Estimated log(det): {log_det_estimate:.6f}")
        print(f"True log(det): {true_log_det:.6f}")
        print(f"Relative error: {relative_error:.2%}")
        
    #Histogram Plot
    display(plot_histogram(dict(list(counts.items())[:15])))
    return log_det_estimate, relative_error

QuantumDeterminantEstimator.run_corrected_simulation = run_corrected_simulation

In [ ]:
def adaptive_parameter_estimation(self, initial_shots=1000): #Appendex D(Impelemntation of adaptive parameter estimation) (Referenced)
    # Start with seed values (smaller than theoretical)
    seed_m = max(4, int(self.precision_bits * 0.7))  # Start with less precision
    seed_shots = min(initial_shots, self.num_samples // 5)
    
    print(f"Starting with seed m={seed_m}, shots={seed_shots}")
    
    #Run initial quantum sampling
    temp_precision = self.precision_bits
    self.precision_bits = seed_m  # Temporarily use seed value
    
    qc = self.build_qpe_circuit()
    display(self.build_qpe_circuit().draw(output='mpl',reverse_bits=True))
    backend = AerSimulator()
    job = backend.run(transpile(qc, backend), shots=seed_shots)
    counts = job.result().get_counts()
    
    # Estimate μ and Δ from actual quantum samples
    eigenvalue_samples = self.process_qpe_results(counts)
    
    if not eigenvalue_samples:
        print("No valid samples, using theoretical parameters")
        self.precision_bits = temp_precision
        return False
    
    # Calculate empirical μ and Δ
    log_samples = [np.log(e) for e in eigenvalue_samples if e > 1e-12]
    if not log_samples:
        print("No positive eigenvalue samples, using theoretical parameters")
        self.precision_bits = temp_precision
        return False
        
    empirical_mu = np.mean(log_samples)
    empirical_delta = np.std(log_samples)
    
    print(f"Empirical μ: {empirical_mu:.4f} (theoretical: {self.mu:.4f})")
    print(f"Empirical Δ: {empirical_delta:.4f} (theoretical: {self.delta:.4f})")
    
    # Check if m is sufficient using paper's formula
    required_m = int(np.ceil(np.log2(4 * self.condition_number / (abs(empirical_mu) * self.target_error))))
    
    if seed_m < required_m:
        print(f"Seed m={seed_m} insufficient, need m≥{required_m}")
        self.precision_bits = max(required_m, temp_precision)
        print(f"Restarting with m={self.precision_bits}")
    else:
        print(f"Seed m={seed_m} is sufficient")
        self.precision_bits = seed_m
    
    # Update N_mc based on empirical Δ
    self.num_samples = max(int(np.ceil((2 * empirical_delta / (abs(empirical_mu) * self.target_error))**2)), 100)
    
    # Update class parameters with empirical values
    self.mu = empirical_mu
    self.delta = empirical_delta
    
    print(f"Updated parameters: m={self.precision_bits}, N_mc={self.num_samples}")
    return True

QuantumDeterminantEstimator.adaptive_parameter_estimation = adaptive_parameter_estimation

In [ ]:
# Get matrix input from user
matrix_input = input("Enter a 2x2 matrix as nested list (ex: [[2.0, 0.3], [0.3, 1.5]]): ")
# Convert string to actual matrix
A_base = np.array(eval(matrix_input))
print("The Matrix A:")
print(A_base)
print(f"Determinant: {np.linalg.det(A_base):.6f}")
print(f"Log(det): {np.log(np.linalg.det(A_base)):.6f}")
print(f"Eigenvalues: {np.linalg.eigvals(A_base)}")
print(f"Condition number: {np.linalg.cond(A_base):.2f}") #K

In [ ]:
# Initialized estimator with maximum 20% target error
targ_error = min(0.2, 0.1)
estimator = QuantumDeterminantEstimator(A_base, target_error=targ_error)

success = estimator.adaptive_parameter_estimation(initial_shots=2000)
if success:
    print("\nAdaptive parameter estimation completed successfully")
else:
    print("\nFalling back to theoretical parameters")

In [ ]:
log_det_estimate, error = estimator.run_corrected_simulation(shots=50000)

In [ ]:
# Compare with classical method and calculate actual complexities

print(f"\n=== SUMARRAY ===")
print(f"\nClassical vs Quantum Comparison")
classical_log_det = np.log(np.linalg.det(A_base))
print(f"Classical log(det): {classical_log_det:.6f}")
print(f"Quantum estimate:   {log_det_estimate:.6f}")
print(f"Difference: {abs(log_det_estimate - classical_log_det):.6f}")

# Calculate complexities
n = estimator.n
kappa = estimator.condition_number
s = min(n, 2)
epsilon = estimator.target_error
log_n = np.log2(n)
log_kappa = np.log2(kappa) if kappa > 1 else 1

# Classical: O(n³)
classical_ops = n**3

# Quantum: O(κ(log κ)²s²/ε³ log n log²(s²/ε))
quantum_ops = kappa * (log_kappa)**2 * s**2 * (1/epsilon**3) * log_n * (np.log2(s**2/epsilon))**2
speedup = classical_ops / quantum_ops if quantum_ops > 0 else 1

print(f"\nComplexity Analysis")
print(f"Classical operations: {classical_ops:,}")
print(f"Quantum operations: {quantum_ops:,.0f}")
